# Assignment 3
## Question 1: Siamese networks & one-shot learning (8pt)
The Cifar-100 dataset is similar to the Cifar-10 dataset. It also consists of 60,000 32x32 RGB images, but they are distributed over 100 classes instead of 10. Thus, each class has much less examples, only 500 training images and 100 testing images per class. For more info about the dataset, see https://www.cs.toronto.edu/~kriz/cifar.html.

*HINT: Import the Cifar-100 dataset directly from Keras, no need to download it from the website. Use* `label_mode="fine"`

### Task 1.1: Siamese network
**a)**
* Train a Siamese Network on the first 80 classes of (the training set of) Cifar-100, i.e. let the network predict the probability that two input images are from the same class. Use 1 as a target for pairs of images from the same class (positive pairs), and 0 for pairs of images from different classes (negative pairs). Randomly select image pairs from Cifar-100, but make sure you train on as many positive pairs as negative pairs.

* Evaluate the performance of the network on 20-way one-shot learning tasks. Do this by generating 250 random tasks and obtain the average accuracy for each evaluation round. Use the remaining 20 classes that were not used for training. The model should perform better than random guessing.

For this question you may ignore the test set of Cifar-100; it suffices to use only the training set and split this, using the first 80 classes for training and the remaining 20 classes for one-shot testing.

*HINT: First sort the data by their labels (see e.g.* `numpy.argsort()`*), then reshape the data to a shape of* `(n_classes, n_examples, width, height, depth)`*, similar to the Omniglot data in Practical 4. It is then easier to split the data by class, and to sample positive and negative images pairs for training the Siamese network.*

*NOTE: do not expect the one-shot accuracy for Cifar-100 to be similar to that accuracy for Omniglot; a lower accuracy can be expected. However, accuracy higher than random guess is certainly achievable.*

In [1]:
# === add code here ===
from keras.layers import Input, Conv2D, Lambda, Dense, Flatten, MaxPooling2D, Dropout, BatchNormalization
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.losses import binary_crossentropy
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

c:\users\illia\appdata\local\conda\conda\envs\tensorflow-gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.datasets import cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

In [3]:
print("X_train shape:", x_train.shape)
print("X_test shape:", x_test.shape)
print("")


y_train

X_train shape: (50000, 32, 32, 3)
X_test shape: (10000, 32, 32, 3)



array([[19],
       [29],
       [ 0],
       ...,
       [ 3],
       [ 7],
       [73]])

In [4]:
sort_p=np.argsort(y_train.flatten(),axis=None)

In [5]:
sort_p[0]

42957

In [6]:
n_sam=[]
cnt=0
lst_id=0
for i in range(50000):
    if(y_train[sort_p[i]]!=lst_id):
        n_sam.append(cnt)
        lst_id=y_train[sort_p[i]]
        cnt=1
    else:
        cnt+=1

In [7]:
for i in range(40000,41000):
    print(y_train[sort_p[i]])

[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]
[80]


In [11]:
n_sam

[500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500]

In [8]:
x_train_new=[x_train[sort_p[i]].flatten() for i in range(50000)]
y_train_new=[y_train[sort_p[i]].flatten() for i in range(50000)]

In [9]:
x_train_new2=np.reshape(x_train_new,(100, 500, 32, 32, 3))
y_train_new2=np.reshape(y_train_new,(100, 500, 1))

In [10]:
print(x_train[sort_p[0]])
print("====")
print(x_train_new2[0][0])

[[[170 174 173]
  [171 175 174]
  [174 178 177]
  ...
  [209 209 209]
  [209 209 209]
  [208 208 208]]

 [[170 174 173]
  [171 175 174]
  [174 178 177]
  ...
  [207 207 207]
  [207 207 207]
  [206 206 206]]

 [[172 176 175]
  [173 177 176]
  [176 180 179]
  ...
  [210 210 210]
  [209 209 209]
  [208 208 208]]

 ...

 [[175 175 175]
  [175 177 176]
  [167 172 168]
  ...
  [183 155 158]
  [219 222 220]
  [218 218 218]]

 [[170 170 170]
  [168 171 171]
  [172 177 176]
  ...
  [218 216 217]
  [220 217 218]
  [215 214 214]]

 [[169 169 169]
  [170 171 171]
  [173 175 175]
  ...
  [219 218 218]
  [216 216 216]
  [213 214 214]]]
====
[[[170 174 173]
  [171 175 174]
  [174 178 177]
  ...
  [209 209 209]
  [209 209 209]
  [208 208 208]]

 [[170 174 173]
  [171 175 174]
  [174 178 177]
  ...
  [207 207 207]
  [207 207 207]
  [206 206 206]]

 [[172 176 175]
  [173 177 176]
  [176 180 179]
  ...
  [210 210 210]
  [209 209 209]
  [208 208 208]]

 ...

 [[175 175 175]
  [175 177 176]
  [167 172 168]

In [11]:
x_train_final=x_train_new2[0:80]
y_train_final=y_train_new2[0:80]
x_test_final=x_train_new2[80:]
y_test_final=y_train_new2[80:]

In [12]:
print(x_train_new2[79])
print('============================')
print(x_train_final[79])
print('============================')
print(y_train_new2[79])
print('============================')
print(y_train_final[79])
print(x_train_final.shape)

[[[[ 44  49  49]
   [ 46  50  47]
   [ 49  51  50]
   ...
   [ 57  58  54]
   [ 51  51  52]
   [ 23  24  28]]

  [[ 47  48  46]
   [ 52  52  47]
   [ 55  53  50]
   ...
   [ 49  50  46]
   [ 42  41  41]
   [ 22  22  25]]

  [[ 52  50  44]
   [ 63  60  53]
   [ 67  62  56]
   ...
   [ 62  61  58]
   [ 49  47  46]
   [ 18  19  21]]

  ...

  [[122 104  91]
   [123 107  93]
   [144 128 115]
   ...
   [104  96  80]
   [110  97  84]
   [110  93  80]]

  [[123 109  98]
   [134 119 108]
   [149 135 124]
   ...
   [107  99  83]
   [112  99  85]
   [122 104  92]]

  [[102  95  89]
   [121 113 106]
   [133 124 117]
   ...
   [113 105  88]
   [112  99  86]
   [124 106  94]]]


 [[[ 46  50  59]
   [ 48  52  63]
   [ 51  53  65]
   ...
   [213 210 204]
   [207 202 193]
   [193 188 180]]

  [[ 47  51  62]
   [ 52  55  67]
   [ 53  56  68]
   ...
   [219 214 206]
   [213 206 199]
   [207 200 193]]

  [[ 51  55  66]
   [ 53  57  68]
   [ 55  58  69]
   ...
   [218 211 204]
   [210 203 195]
   [206 199

In [14]:
#c_train={a:[500*(a), 500*(a+1)-1] for a in range(80)}
c_train={a:a for a in range(80)}
c_test={a:a for a in range(80,100)}

print("X_train shape:", x_train_final.shape)
print("")
print("training alphabets")
print([key for key in c_train.keys()])

c_test

X_train shape: (80, 500, 32, 32, 3)

training alphabets
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79]


{80: 80,
 81: 81,
 82: 82,
 83: 83,
 84: 84,
 85: 85,
 86: 86,
 87: 87,
 88: 88,
 89: 89,
 90: 90,
 91: 91,
 92: 92,
 93: 93,
 94: 94,
 95: 95,
 96: 96,
 97: 97,
 98: 98,
 99: 99}

In [15]:
def get_batch(batch_size, X):
    """Create batch of n pairs, half same class, half different class"""
    n_classes, n_examples, w, h, d = X.shape
    #n_examples, w, h, d = X.shape
    # randomly sample several classes to use in the batch
    categories = np.random.choice(n_classes, size=(batch_size,), replace=False)
    # initialize 2 empty arrays for the input image batch
    pairs = [np.zeros((batch_size, h, w, d)) for i in range(2)]
    # initialize vector for the targets, and make one half of it '1's, so 2nd half of batch has same class
    targets = np.zeros((batch_size,))
    targets[batch_size//2:] = 1
    for i in range(batch_size):
        category = categories[i]
        idx_1 = np.random.randint(0, n_examples)
        pairs[0][i, :, :, :] = X[category, idx_1].reshape(w, h, d)
        idx_2 = np.random.randint(0, n_examples)
        # pick images of same class for 1st half, different for 2nd
        if i >= batch_size // 2:
            category_2 = category
        else:
            #add a random number to the category modulo n_classes to ensure 2nd image has different category
            category_2 = (category + np.random.randint(1,n_classes)) % n_classes
        pairs[1][i, :, :, :] = X[category_2,idx_2].reshape(w, h, d)
    return pairs, targets

def batch_generator(batch_size, X):
    """a generator for batches, so model.fit_generator can be used. """
    while True:
        pairs, targets = get_batch(batch_size, X)
        yield (pairs, targets)

def train(model, X_train, batch_size=32, steps_per_epoch=100, epochs=1):
    model.fit_generator(batch_generator(batch_size, X_train), steps_per_epoch=steps_per_epoch, epochs=epochs)

In [16]:
input_shape = (32, 32, 3)
left_input = Input(input_shape)
right_input = Input(input_shape)

# build convnet to use in each siamese 'leg'
convnet = Sequential()
convnet.add(Conv2D(64, (5,5), activation='relu', input_shape=input_shape, kernel_regularizer=l2(2e-4)))
convnet.add(MaxPooling2D())
convnet.add(BatchNormalization())
convnet.add(Dropout(0.25))
convnet.add(Conv2D(128, (3,3), activation='relu', kernel_regularizer=l2(2e-4)))
convnet.add(MaxPooling2D())
convnet.add(BatchNormalization())
convnet.add(Dropout(0.25))
convnet.add(Conv2D(128, (2,2), activation='relu', kernel_regularizer=l2(2e-4)))
convnet.add(MaxPooling2D())
convnet.add(BatchNormalization())
convnet.add(Dropout(0.25))
convnet.add(Conv2D(256, (2,2), activation='relu', kernel_regularizer=l2(2e-4)))
convnet.add(Flatten())
convnet.add(BatchNormalization())
convnet.add(Dropout(0.25))
convnet.add(Dense(4096, activation="sigmoid", kernel_regularizer=l2(1e-3)))
convnet.summary()

# encode each of the two inputs into a vector with the convnet
encoded_l = convnet(left_input)
encoded_r = convnet(right_input)

# merge two encoded inputs with the L1 distance between them, and connect to prediction output layer
L1_distance = lambda x: K.abs(x[0]-x[1])
both = Lambda(L1_distance)([encoded_l, encoded_r])
prediction = Dense(1, activation='sigmoid')(both)
siamese_net = Model(inputs=[left_input,right_input], outputs=prediction)


siamese_net.compile(loss="binary_crossentropy", optimizer="adam")

siamese_net.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 64)        4864      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 64)        256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 6, 6, 128)         512       
__________

In [23]:
def make_oneshot_task(N, X, c, language=None):
    """Create pairs of (test image, support set image) with ground truth, for testing N-way one-shot learning."""
    n_classes, n_examples, w, h, d = X.shape
    indices = np.random.randint(0, n_examples, size=(N,))
    #if language is not None:
        #low, high = c[language]
        #if N > high - low:
        #    raise ValueError("This language ({}) has less than {} letters".format(language, N))
    #    categories = np.random.choice(range(low,high), size=(N,), replace=False)
    #else:  # if no language specified just pick a bunch of random letters
    categories = np.random.choice(range(n_classes), size=(N,), replace=False)    
    #print(categories)
    true_category = categories[0]
    ex1, ex2 = np.random.choice(n_examples, replace=False, size=(2,))
    test_image = np.asarray([X[true_category, ex1, :, :]]*N).reshape(N, w, h, d)
    support_set = X[categories, indices, :, :]
    support_set[0, :, :] = X[true_category, ex2]
    support_set = support_set.reshape(N, w, h, d)
    targets = np.zeros((N,))
    targets[0] = 1
    targets, test_image, support_set = shuffle(targets, test_image, support_set)
    pairs = [test_image, support_set]
    return pairs, targets

def test_oneshot(model, X, c, N=20, k=250, language=None, verbose=True):
    """Test average N-way oneshot learning accuracy of a siamese neural net over k one-shot tasks."""
    n_correct = 0
    if verbose:
        print("Evaluating model on {} random {}-way one-shot learning tasks ...".format(k, N))
    for i in range(k):
        inputs, targets = make_oneshot_task(N, X, c, language=language)
        probs = model.predict(inputs)
        if np.argmax(probs) == np.argmax(targets):
            n_correct += 1
    percent_correct = (100.0*n_correct / k)
    if verbose:
        print("Got an average of {}% accuracy for {}-way one-shot learning".format(percent_correct, N))
    return percent_correct

In [24]:
def concat_images(X):
    """Concatenates a bunch of images into a big matrix for plotting purposes."""
    nc,h,w,_ = X.shape
    X = X.reshape(nc,h,w)
    n = np.ceil(np.sqrt(nc)).astype("int8")
    img = np.zeros((n*w,n*h))
    x = 0
    y = 0
    for example in range(nc):
        img[x*w:(x+1)*w,y*h:(y+1)*h] = X[example]
        y += 1
        if y >= n:
            y = 0
            x += 1
    return img

def plot_oneshot_task(pairs):
    """Takes a one-shot task given to a siamese net and  """
    fig,(ax1,ax2) = plt.subplots(2)
    ax1.matshow(pairs[0][0].reshape(105,105),cmap='gray')
    img = concat_images(pairs[1])
    ax1.get_yaxis().set_visible(False)
    ax1.get_xaxis().set_visible(False)
    ax2.matshow(img,cmap='gray')
    plt.xticks([])
    plt.yticks([])
    plt.show()

In [25]:
pairs, targets = make_oneshot_task(20, x_train_final, c_train, language=5)
#plot_oneshot_task(pairs)

In [20]:
import h5py
#import h5py_cache as h5c
h5py.run_tests()

.....................................................x...............................s...................................x....................................s...s......ss.ss............................................................................ss...................ssssss..............................F..................................x....x........................x.....x.................................................ssss.........F....
FAIL: test_out_of_order_offsets (h5py.tests.old.test_h5t.TestCompound)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\users\illia\appdata\local\conda\conda\envs\tensorflow-gpu\lib\site-packages\h5py\tests\old\test_h5t.py", line 61, in test_out_of_order_offsets
    self.assertEqual(tid.dtype, expected_dtype)
AssertionError: dtype[13 chars]1','f3','f2'], 'formats':['<f4','<f8','<i4'], [30 chars]:20}) != dtype[13 chars]1','f2','f3'], 'formats':['<f4','<i4','<f8'], [30 chars]:20})



<unittest.runner.TextTestResult run=446 errors=0 failures=2>

In [29]:
loops = 250
best_acc = 0
for i in range(loops):
    print("=== Training loop {} ===".format(i+1))
    train(siamese_net, x_train_final)
    test_acc = test_oneshot(siamese_net, x_test_final, c_test)
    if test_acc >= best_acc:
        print("New best one-shot accuracy, saving model ...")
        siamese_net.save(os.path.join("models", "siamese_omniglot.h5"))
        best_acc = test_acc

=== Training loop 1 ===
Epoch 1/1
100/100 [==============================] - 4s 36ms/step - loss: 0.6819
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 12.8% accuracy for 20-way one-shot learning
New best one-shot accuracy, saving model ...
=== Training loop 2 ===
Epoch 1/1
100/100 [==============================] - 3s 34ms/step - loss: 0.6797
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 12.0% accuracy for 20-way one-shot learning
=== Training loop 3 ===
Epoch 1/1
100/100 [==============================] - 3s 33ms/step - loss: 0.6899
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 12.8% accuracy for 20-way one-shot learning
New best one-shot accuracy, saving model ...
=== Training loop 4 ===
Epoch 1/1
100/100 [==============================] - 3s 34ms/step - loss: 0.6803
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 11.2% accuracy for 20-

100/100 [==============================] - 3s 33ms/step - loss: 0.6746
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 12.4% accuracy for 20-way one-shot learning
=== Training loop 36 ===
Epoch 1/1
100/100 [==============================] - 3s 33ms/step - loss: 0.6817
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 11.6% accuracy for 20-way one-shot learning
=== Training loop 37 ===
Epoch 1/1
100/100 [==============================] - 3s 33ms/step - loss: 0.6812
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 11.6% accuracy for 20-way one-shot learning
=== Training loop 38 ===
Epoch 1/1
100/100 [==============================] - 3s 33ms/step - loss: 0.6899
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 10.0% accuracy for 20-way one-shot learning
=== Training loop 39 ===
Epoch 1/1
100/100 [==============================] - 3s 33ms/step - loss: 

100/100 [==============================] - 3s 33ms/step - loss: 0.6715
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 12.0% accuracy for 20-way one-shot learning
=== Training loop 71 ===
Epoch 1/1
100/100 [==============================] - 3s 34ms/step - loss: 0.6658
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 11.2% accuracy for 20-way one-shot learning
=== Training loop 72 ===
Epoch 1/1
100/100 [==============================] - 3s 33ms/step - loss: 0.6769
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 12.0% accuracy for 20-way one-shot learning
=== Training loop 73 ===
Epoch 1/1
100/100 [==============================] - 3s 33ms/step - loss: 0.6838
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 14.4% accuracy for 20-way one-shot learning
=== Training loop 74 ===
Epoch 1/1
100/100 [==============================] - 3s 33ms/step - loss: 

100/100 [==============================] - 3s 33ms/step - loss: 0.6756
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 14.8% accuracy for 20-way one-shot learning
=== Training loop 106 ===
Epoch 1/1
100/100 [==============================] - 3s 33ms/step - loss: 0.6858
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 11.6% accuracy for 20-way one-shot learning
=== Training loop 107 ===
Epoch 1/1
100/100 [==============================] - 3s 33ms/step - loss: 0.6732
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 8.0% accuracy for 20-way one-shot learning
=== Training loop 108 ===
Epoch 1/1
100/100 [==============================] - 3s 33ms/step - loss: 0.6656
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 13.2% accuracy for 20-way one-shot learning
=== Training loop 109 ===
Epoch 1/1
100/100 [==============================] - 3s 34ms/step - los

100/100 [==============================] - 3s 33ms/step - loss: 0.6758
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 12.8% accuracy for 20-way one-shot learning
=== Training loop 141 ===
Epoch 1/1
100/100 [==============================] - 3s 33ms/step - loss: 0.6784
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 14.8% accuracy for 20-way one-shot learning
=== Training loop 142 ===
Epoch 1/1
100/100 [==============================] - 3s 33ms/step - loss: 0.6665
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 9.6% accuracy for 20-way one-shot learning
=== Training loop 143 ===
Epoch 1/1
100/100 [==============================] - 3s 33ms/step - loss: 0.6681
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 15.6% accuracy for 20-way one-shot learning
=== Training loop 144 ===
Epoch 1/1
100/100 [==============================] - 3s 33ms/step - los

100/100 [==============================] - 3s 33ms/step - loss: 0.6650
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 14.8% accuracy for 20-way one-shot learning
=== Training loop 176 ===
Epoch 1/1
100/100 [==============================] - 3s 33ms/step - loss: 0.6807
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 14.4% accuracy for 20-way one-shot learning
=== Training loop 177 ===
Epoch 1/1
100/100 [==============================] - 3s 34ms/step - loss: 0.6729
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 14.4% accuracy for 20-way one-shot learning
=== Training loop 178 ===
Epoch 1/1
100/100 [==============================] - 3s 34ms/step - loss: 0.6716
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 15.6% accuracy for 20-way one-shot learning
=== Training loop 179 ===
Epoch 1/1
100/100 [==============================] - 3s 34ms/step - lo

100/100 [==============================] - 3s 33ms/step - loss: 0.6667
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 12.0% accuracy for 20-way one-shot learning
=== Training loop 211 ===
Epoch 1/1
100/100 [==============================] - 3s 34ms/step - loss: 0.6640
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 14.4% accuracy for 20-way one-shot learning
=== Training loop 212 ===
Epoch 1/1
100/100 [==============================] - 3s 33ms/step - loss: 0.6604
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 12.4% accuracy for 20-way one-shot learning
=== Training loop 213 ===
Epoch 1/1
100/100 [==============================] - 3s 33ms/step - loss: 0.6701
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 18.8% accuracy for 20-way one-shot learning
=== Training loop 214 ===
Epoch 1/1
100/100 [==============================] - 3s 34ms/step - lo

100/100 [==============================] - 3s 34ms/step - loss: 0.6685
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 15.2% accuracy for 20-way one-shot learning
=== Training loop 246 ===
Epoch 1/1
100/100 [==============================] - 3s 34ms/step - loss: 0.6655
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 17.6% accuracy for 20-way one-shot learning
=== Training loop 247 ===
Epoch 1/1
100/100 [==============================] - 3s 34ms/step - loss: 0.6694
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 14.4% accuracy for 20-way one-shot learning
=== Training loop 248 ===
Epoch 1/1
100/100 [==============================] - 3s 34ms/step - loss: 0.6689
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 16.4% accuracy for 20-way one-shot learning
=== Training loop 249 ===
Epoch 1/1
100/100 [==============================] - 3s 34ms/step - lo

In [30]:
print('Best acc: ',best_acc)

Best acc:  22.4


In [26]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

['/device:CPU:0', '/device:GPU:0']


***

**b)** Briefly motivate your model's architecture, as well as its performance. What accuracy would random guessing achieve (on average)?

**Answer:**

*=== write your answer here ===*

***

**c)** Compare the performance of your Siamese network for Cifar-100 to the Siamese network from Practical 4 for Omniglot. Name three fundamental differences between the Cifar-100 and Omniglot datasets. How do these differences influence the difference in one-shot accuracy?

**Answer:**

*=== write your answer here ===*

***

### Task 1.2: One-shot learning with neural codes
**a)**
* Train a CNN classifier on the first 80 classes of Cifar-100. Make sure it achieves at least 40% classification accuracy on those 80 classes (use the test set to validate this accuracy).
* Then use neural codes from one of the later hidden layers of the CNN with L2-distance to evaluate one-shot learning accuracy for the remaining 20 classes of Cifar-100. I.e. for a given one-shot task, obtain neural codes for the test image as well as the support set. Then pick the image from the support set that is closest (in L2-distance) to the test image as your one-shot prediction.

In [ ]:
# === add code here ===

***

**b)** Briefly motivate your CNN architecture, and discuss the difference in one-shot accuracy between the Siamese network approach and the CNN neural codes approach.

**Answer:**

*=== write your answer here ===*